# Korean LLM (Large Language Model) Inference
---

### Model: [KKULM-Polyglot-12.8B](https://huggingface.co/nlpai-lab/kullm-polyglot-12.8b-v2)

- LLM GitHub: https://github.com/nlpai-lab/KULLM
- Hugging Face model hub: https://huggingface.co/nlpai-lab/kullm-polyglot-12.8b-v2

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates') 

In [2]:
!pip install -qU boto3 huggingface_hub sagemaker langchain deepspeed 
!pip install -qU bitsandbytes accelerate peft

In [3]:
import os
from pathlib import Path
from huggingface_hub import snapshot_download

# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path("/home/ec2-user/SageMaker/hf_cache")
local_model_path.mkdir(exist_ok=True)
model_id = "nlpai-lab/kullm-polyglot-12.8b-v2"

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_id,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [4]:
import torch
from transformers import BitsAndBytesConfig

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
import torch
import deepspeed
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, GPTNeoXLayer

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    cache_dir=local_model_path,
    quantization_config=bnb_config
)

[2023-07-23 13:54:54,538] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
# config = {
#     "tensor_parallel": {"tp_size": 1},
#     "dtype": "fp16",
#     "injection_policy": {
#         GPTNeoXLayer:('attention.dense', 'mlp.dense_4h_to_h')
#     }
# }

# modl = deepspeed.init_inference(model, config)

local_rank = int(os.getenv('LOCAL_RANK', '0'))
tokenizer = AutoTokenizer.from_pretrained(model_id)
generator = pipeline(
    task="text-generation", model=model, tokenizer=tokenizer
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [8]:
import json
from inference_lib import Prompter
prompter = Prompter("kullm")

params = {
    "do_sample": False,
    "max_new_tokens": 256,
    "return_full_text": True,
    "temperature": 0.01,
    "top_p": 0.9,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "presence_penalty": None,
    "eos_token_id": 2,
}

instruction = "아래 질문에 100글자 이상으로 자세하게 대답해줘."
#instruction = ""
input_text = "고려대학교에 대해서 알려줘"
prompt = prompter.generate_prompt(instruction, input_text)
payload = {
    "inputs": [prompt,],
    "parameters": params
}

In [9]:
text_inputs, params = payload["inputs"], payload["parameters"]
result = generator(text_inputs, **params)
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[[{'generated_text': '고려대학교는 한국의 명문 사립대학으로, 1931년 5월 1일에 설립되었습니다. 고려대학교는 한국에서 가장 오래된 고등 교육 기관 중 하나로, 한국의 역사와 문화적 전통을 이어가고 있습니다. 고려대학교는 학문과 연구 분야에서 뛰어난 성과를 거두며 한국 사회 발전에 기여하고 있으며, 다양한 분야에서 활약하는 동문들을 배출했습니다. 고려대학교는 학생들이 자신의 잠재력을 최대한 발휘할 수 있도록 지원하기 위해 최선을 다하고 있으며, 학생들에게 다양한 기회와 자원을 제공합니다.'}]]
